<a href="https://colab.research.google.com/github/Us9r/starlink-sentiment-analysis/blob/main/starlinkSentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Data Cleaning
import, load dataset dan, pre-processing text

## Import, load

In [1]:
!pip install Sastrawi
!pip install googletrans==4.0.0-rc1 httpx>=0.23.0
!pip install tqdm
!pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langsmith 0.3.33 requires httpx<1,>=0.23.0, but you have httpx 0.13.3 which is incompatible.
google-genai 1.11.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.13.3 which is incompatible.
openai 1.75.0 requires httpx<1,>=0.23.0, but you have httpx 0.13.3 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.StopWordRemover.StopWordRemover import StopWordRemover
from Sastrawi.Dictionary.ArrayDictionary import ArrayDictionary
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from googletrans import Translator
from tqdm import tqdm

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Us9r/starlink-sentiment-analysis/main/comments_with_replies%20(april).csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10857 entries, 0 to 10856
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Timestamp  10857 non-null  object
 1   Username   10856 non-null  object
 2   VideoID    10857 non-null  object
 3   Comment    10857 non-null  object
 4   Date       10857 non-null  object
dtypes: object(5)
memory usage: 424.2+ KB


In [ ]:
df_comments = df.copy()
df_comments = df_comments[['Username','Comment']]

## Dropna, Drop duplicates

In [ ]:
df_comments = df_comments.dropna()

In [ ]:
df_comments = df_comments.drop_duplicates()

In [ ]:
df_comments.head(3)

,Username,Comment
0,@AndreiSurya-n7f,Langkah langkah buat paket internet Gimna
1,@Edan27271,Punya elon musk❎\nPunya mas David✅
2,@littentamba4359,"Bng, bs spill powerbengny \nTrus beli d mn pow..."


## Case Folding

In [ ]:
# Create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def casefolding(text):
    text = text.lower()
    return text

df_comments['Comment'] = df_comments['Comment'].apply(casefolding)

## Removal Special Characters

Menghapus Angka
Menghapus Tanda Baca
Menghapus White Space
Menggunakan strip() untuk Menghapus Whitespace di Awal dan Akhir
Menggunakan replace() untuk Menghapus Whitespace di Seluruh String

In [ ]:
def remove_special_characters(text):
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.strip()  # Remove whitespace at the beginning and end
    text = text.replace(" ", " ") # Remove all whitespace (if you want to replace multiple spaces with a single space)
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention dan special characters
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka

    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
    return text

df_comments['Comment'] = df_comments['Comment'].apply(remove_special_characters)

In [ ]:
hapus_iklan = ["cair", "anti mangkir", "depo", "gacor", "wede", "wd", "scatter", "rungkat","javaland","s ago"]

# Gabungkan string dengan operator '|' (OR)
pola = '|'.join(hapus_iklan)

jumlah_baris_dihapus = df_comments['Comment'].str.contains(pola, case=False).sum()
jumlah_baris_dihapus

np.int64(68)

In [ ]:
df_comments = df_comments[~df_comments['Comment'].str.contains(pola, case=False)] #menghapus iklan

##  Stopword Removal (Filtering)

In [ ]:
from collections import Counter
# identifikasi
# Gabungkan semua komentar menjadi satu string panjang
all_text = ' '.join(df_comments['Comment'].tolist())

# Hitung frekuensi kata
word_counts = Counter(word_tokenize(all_text))

# Tampilkan kata-kata yang paling sering muncul
print(word_counts.most_common(20)) # Menampilkan 20 kata teratas

[('di', 3273), ('bisa', 1883), ('bang', 1825), ('starlink', 1654), ('yg', 1651), ('ada', 1447), ('ini', 1433), ('buat', 1347), ('internet', 1110), ('yang', 1106), ('dan', 1061), ('nya', 1051), ('itu', 970), ('ya', 857), ('gak', 833), ('ga', 806), ('untuk', 795), ('kalo', 781), ('ke', 758), ('kalau', 666)]


In [ ]:
# Definisikan stopword custom
stopword_custom = ["hadir", "first", "lah", "pertama","ada", "ini",'iya','yaa','nya','na','sih','ku',"di","ya","loh","kah","woi","woii","woy","p","nihh","s ago","pertamaxx", "d", "mn", "trus", "dri", "kn", "kan", "bg", "bng", "bang", "mas","bs", "david", "brendi", "david brendi", "yg", "yng", "yang", "nya", "ya", "y", "kalo", "aja", "sih","bisa"]

# Gabungkan stopword bawaan dan custom
stop_words = StopWordRemoverFactory().get_stop_words() + stopword_custom

# Buat objek StopWordRemover
new_array = ArrayDictionary(stop_words)
stop_words_remover_new = StopWordRemover(new_array)

# Fungsi untuk menghapus stopwords
def stopword(str_text):
    str_text = stop_words_remover_new.remove(str_text)
    return str_text

# Terapkan pada kolom 'Comment'
df_comments['Comment_NoStopword'] = df_comments['Comment'].apply(lambda x: stopword(x))

In [ ]:
# Identifikasi dan hapus baris kosong setelah stopword removal
df_comments = df_comments[df_comments['Comment'] != '']
df_comments = df_comments[df_comments['Comment'].str.strip() != ''] #tambahan untuk menghapus spasi kosong

## Tokenizing
with end result of df_tokenized

In [ ]:
def tokenize_text(text):
    # Normalisasi kata-kata informal
    normalization_dict = {
        "gmna": "gimana",
        "gk": "tidak",
        "gak": "tidak",
        "ga": "tidak",
        "gaa": "tidak",
        "engga": "tidak",
        "kl": "mungkin",
        "klo": "mungkin",
        "hrga": "harga",
        "dpt": "dapat",
        "powerbengny": "power bank",
        "powerbngny": "power bank",
        "powerbankny": "power bank",
        "starbak": "starlink",
        "bs":"bisa"
    }

    # Tokenisasi dengan split dan normalisasi
    tokens = []
    for word in text.split():
        if word in normalization_dict:
            tokens.append(normalization_dict[word])
        else:
            tokens.append(word)

    return tokens

df_comments['Comments_Tokens'] = df_comments['Comment_NoStopword'].apply(lambda x: tokenize_text(x))

In [ ]:
df_tokenized = df_comments[df_comments['Comments_Tokens'].map(lambda d: len(d)) > 0]

#### saving df_tokenized

In [ ]:
df_tokenized.to_csv('df_tokenized.csv', index=False)

## Stemming
Elapsed for 58 minutes to generate df_stemmed

In [ ]:
tqdm.pandas() #progress bar

def stemming(text_tokens):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    stemmed_tokens = [stemmer.stem(token) for token in text_tokens]
    return stemmed_tokens

# Terapkan stemming dengan progress bar
df_tokenized['Comments_Stemming'] = df_tokenized['Comments_Tokens'].progress_apply(lambda x: stemming(x))

100%|██████████| 10604/10604 [58:45<00:00,  3.01it/s]
<ipython-input-22-8a4b5a5b07b8>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tokenized['Comments_Stemming'] = df_tokenized['Comments_Tokens'].progress_apply(lambda x: stemming(x))


In [ ]:
df_stemmed = df_tokenized.copy()
df_stemmed.to_csv('df_stemmed.csv', index=False)

In [ ]:
df_stemmed.head(3)

,Username,Comment,Comment_NoStopword,Comments_Tokens,Comments_Stemming
0,@AndreiSurya-n7f,langkah langkah buat paket internet gimna,langkah langkah buat paket internet gimna,"[langkah, langkah, buat, paket, internet, gimna]","[langkah, langkah, buat, paket, internet, gimna]"
1,@Edan27271,punya elon musk punya mas david,punya elon musk punya david,"[punya, elon, musk, punya, david]","[punya, elon, musk, punya, david]"
2,@littentamba4359,bng bs spill powerbengny trus beli d mn power...,bs spill powerbengny beli mn powerbngny bng m...,"[bisa, spill, power bank, beli, mn, power bank...","[bisa, spill, power bank, beli, mn, power bank..."


## Translating
Elapsed For 2 Hours 13 minutes to generate df_translated

In [ ]:
# jika diperlukan
df_stemmed = pd.read_csv('df_stemmed.csv')
df_stemmed = df_stemmed['Comments_Stemming']

"\njika diperlukan\ndf_stemmed = pd.read_csv('df_stemmed.csv')\ndf_stemmed = df_stemmed['Comments_Stemming']\n"

In [ ]:
translator = Translator()

def translate_comment(comment_tokens):
    try:
        # Gabungkan token menjadi satu string sebelum diterjemahkan
        comment_text = " ".join(comment_tokens)
        translation = translator.translate(comment_text, dest='en').text
        return translation
    except Exception as e:
        print(f"Translation error for comment '{comment_text}': {e}")
        return comment_text

# Terapkan terjemahan pada kolom 'Comments_Stemming' di df_stemmed
df_stemmed['Translated_Comment'] = df_stemmed['Comments_Stemming'].progress_apply(translate_comment)

# Simpan hasil terjemahan di df_translated
df_translated = df_stemmed[['Translated_Comment']].copy()

  3%|▎         | 324/10604 [04:05<1:05:08,  2.63it/s]

Translation error for comment 'rada kurang masih lumayan speedny': the JSON object must be str, bytes or bytearray, not NoneType
Translation error for comment 'serius nanya kalau bawa traveling guna pesawat kah bawa': the JSON object must be str, bytes or bytearray, not NoneType


 36%|███▌      | 3831/10604 [48:28<1:37:33,  1.16it/s]

Translation error for comment '   ': sequence item 0: expected str instance, NoneType found


 61%|██████    | 6490/10604 [1:22:02<30:13,  2.27it/s]

Translation error for comment 'kalau orang kaya tinggal pelosok tidak internet fiber sinyal semua operator edge perangkat sangat cocok buat': the JSON object must be str, bytes or bytearray, not NoneType
Translation error for comment 'tidak cocok duit': the JSON object must be str, bytes or bytearray, not NoneType


 92%|█████████▏| 9754/10604 [2:03:47<07:25,  1.91it/s]

Translation error for comment 'kayak operator kuasa diindo tidak bakal diam deh entah nuntut harga paket starlink dinaikin apa': the JSON object must be str, bytes or bytearray, not NoneType


100%|██████████| 10604/10604 [2:13:57<00:00,  1.32it/s]


#2. Vader Sentiment Analysis
Sentimen Keseluruhan Positif: Nilai Overall Sentiment adalah "Positive", yang menandakan bahwa sentimen positif mendominasi dalam kumpulan data komentar yang dianalisis.

> * 'Overall Sentiment': 'Positive',
> * 'Positive Count': 3342,
> * 'Negative Count': 1703,
> * 'Neutral Count': 5558,
> * 'Positive Percentage': 31.519381307177213,
> * 'Negative Percentage': 16.06149203055739,
> * 'Neutral Percentage': 52.41912666226539}

In [5]:
# jika diperlukan
df_translated = pd.read_csv('https://raw.githubusercontent.com/Us9r/starlink-sentiment-analysis/main/df_translated.csv')

Hapus sisa eror translation

In [6]:
error_values = [
    'rada kurang masih lumayan speedny',
    'serius nanya kalau bawa traveling guna pesawat kah bawa',
    '   ',
    'kalau orang kaya tinggal pelosok tidak internet fiber sinyal semua operator edge perangkat sangat cocok buat',
    'tidak cocok duit',
    'kayak operator kuasa diindo tidak bakal diam deh entah nuntut harga paket starlink dinaikin apa'
]

# Filter rows containing error values in 'Comments_Stemming' column
filtered_df = df_translated[~df_translated['Translated_Comment'].apply(lambda x: any(val in x for val in error_values))]

## Labeling

In [7]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

tqdm.pandas()

analyzer = SentimentIntensityAnalyzer()

def analyze_sentiment(text):
    scores = analyzer.polarity_scores(text)
    return scores

df_translated['Sentiment_Scores'] = df_translated['Translated_Comment'].progress_apply(analyze_sentiment)

100%|██████████| 10603/10603 [00:01<00:00, 7722.06it/s]


## Dividing result into each collumn

In [8]:
df_translated = pd.concat([df_translated.drop(['Sentiment_Scores'], axis=1), df_translated['Sentiment_Scores'].apply(pd.Series)], axis=1)

In [9]:
def analyze_sentiment_results(df_translated):
    # Add a new column 'Sentiment' to store the sentiment labels
    df_sentiment = df_translated.copy()
    df_sentiment['Sentiment'] = ''

    # Apply sentiment classification based on 'compound' score
    df_sentiment.loc[df_sentiment['compound'] >= 0.05, 'Sentiment'] = 'Positive'
    df_sentiment.loc[df_sentiment['compound'] <= -0.05, 'Sentiment'] = 'Negative'
    df_sentiment.loc[df_sentiment['compound'].between(-0.05, 0.05), 'Sentiment'] = 'Neutral'

    # Calculate the overall sentiment
    overall_sentiment = 'Neutral'
    if df_sentiment['compound'].mean() >= 0.05: # Changed df_translated to df_sentiment
        overall_sentiment = 'Positive'
    elif df_sentiment['compound'].mean() <= -0.05: # Changed df_translated to df_sentiment
        overall_sentiment = 'Negative'

    # Count the number of positive, negative, and neutral sentiments
    positive_count = df_sentiment[df_sentiment['Sentiment'] == 'Positive'].shape[0] # Changed df_translated to df_sentiment
    negative_count = df_sentiment[df_sentiment['Sentiment'] == 'Negative'].shape[0] # Changed df_translated to df_sentiment
    neutral_count = df_sentiment[df_sentiment['Sentiment'] == 'Neutral'].shape[0] # Changed df_translated to df_sentiment

    # Calculate percentages
    total_count = df_sentiment.shape[0] # Changed df_translated to df_sentiment
    positive_percentage = (positive_count / total_count) * 100 if total_count > 0 else 0
    negative_percentage = (negative_count / total_count) * 100 if total_count > 0 else 0
    neutral_percentage = (neutral_count / total_count) * 100 if total_count > 0 else 0

    analysis = {
        'Overall Sentiment': overall_sentiment,
        'Positive Count': positive_count,
        'Negative Count': negative_count,
        'Neutral Count': neutral_count,
        'Positive Percentage': positive_percentage,
        'Negative Percentage': negative_percentage,
        'Neutral Percentage': neutral_percentage,
    }

    return analysis, df_sentiment  # Return both the analysis and the updated DataFrame

analysis_results, df_sentiment = analyze_sentiment_results(df_translated)

In [10]:
df_sentiment

,Translated_Comment,neg,neu,pos,compound,Sentiment
0,steps to make a gimna internet package,0.000,1.000,0.000,0.0000,Neutral
1,Have Elon Musk Has David,0.000,1.000,0.000,0.0000,Neutral
2,Can Spill Power Bank Buy Mn Power Bank BNG Mn ...,0.000,1.000,0.000,0.0000,Neutral
3,Pay which month,0.412,0.588,0.000,-0.1027,Negative
4,at the same time WiFi how,0.000,1.000,0.000,0.0000,Neutral
...,...,...,...,...,...,...
10598,greeted me,0.000,0.323,0.677,0.2732,Positive
10599,nih,0.000,1.000,0.000,0.0000,Neutral
10600,number one electoral district ddddd,0.000,0.755,0.245,0.0772,Positive
10601,Stralink kelling,0.000,1.000,0.000,0.0000,Neutral


## Result
df_sentiment

In [ ]:
analysis_results

{'Overall Sentiment': 'Positive',
 'Positive Count': 3342,
 'Negative Count': 1703,
 'Neutral Count': 5558,
 'Positive Percentage': 31.519381307177213,
 'Negative Percentage': 16.06149203055739,
 'Neutral Percentage': 52.41912666226539}

In [11]:
df_sentiment.to_csv('df_sentiment.csv', index=False)

# 3. Training ML model

In [12]:
#jika diperlukan
df_sentiment = pd.read_csv('https://raw.githubusercontent.com/Us9r/starlink-sentiment-analysis/main/df_sentiment.csv')

In [32]:
df_sentiment_selected = df_sentiment[['Translated_Comment', 'Sentiment']]

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Assuming df_sentiment is your DataFrame with 'Translated_Comment' and 'Sentiment' columns

# Prepare the data
X = df_sentiment_selected['Translated_Comment']
y = df_sentiment_selected['Sentiment']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Transform the testing data
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Train a Multinomial Naive Bayes classifier
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tfidf, y_train)

# Make predictions on the test set
y_pred = naive_bayes_classifier.predict(X_test_tfidf)

# Evaluate the model
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

    Negative       0.89      0.11      0.19       321
     Neutral       0.73      0.95      0.82      1128
    Positive       0.75      0.69      0.72       672

    accuracy                           0.74      2121
   macro avg       0.79      0.58      0.58      2121
weighted avg       0.76      0.74      0.69      2121



In [34]:
# Make predictions on the training set
y_train_pred = naive_bayes_classifier.predict(X_train_tfidf)

# Evaluate the model on the training set
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

    Negative       0.98      0.23      0.37      1382
     Neutral       0.79      0.99      0.88      4430
    Positive       0.85      0.83      0.84      2670

    accuracy                           0.82      8482
   macro avg       0.87      0.68      0.70      8482
weighted avg       0.84      0.82      0.78      8482



Nilai rendah, diputuskan untuk menggunakan model Deep Learning

# 4. Deep Learning Model
menggunakan LSTM berhasil memperoleh
> akurasi training 0.97
> akurasi testing 0.86

In [37]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

# Prepare the data
X = df_sentiment_selected['Translated_Comment']
y = df_sentiment_selected['Sentiment']

# Convert sentiment labels to numerical values
sentiment_mapping = {'Positive': 0, 'Negative': 1, 'Neutral': 2}
y = y.map(sentiment_mapping)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize the text data
tokenizer = Tokenizer(num_words=5000) # Adjust num_words as needed
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to have the same length
max_sequence_length = 100 # Adjust as needed
X_train_padded = pad_sequences(X_train_seq, maxlen=max_sequence_length)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_sequence_length)

# Define the model
model = Sequential()
model.add(Embedding(5000, 128, input_length=max_sequence_length))
model.add(LSTM(64))
model.add(Dense(3, activation='softmax')) # 3 output neurons for 3 sentiment classes

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train, epochs=5, batch_size=32, validation_split=0.2) # Adjust epochs and batch_size as needed

# Evaluate the model
y_pred_prob = model.predict(X_test_padded)
y_pred = y_pred_prob.argmax(axis=1) # Convert probabilities to class labels
print(classification_report(y_test, y_pred))


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


213/213 ━━━━━━━━━━━━━━━━━━━━ 22s 88ms/step - accuracy: 0.5794 - loss: 0.9140 - val_accuracy: 0.7778 - val_loss: 0.5780
Epoch 2/5
213/213 ━━━━━━━━━━━━━━━━━━━━ 19s 82ms/step - accuracy: 0.8579 - loss: 0.3961 - val_accuracy: 0.8326 - val_loss: 0.4552
Epoch 3/5
213/213 ━━━━━━━━━━━━━━━━━━━━ 20s 81ms/step - accuracy: 0.9167 - loss: 0.2592 - val_accuracy: 0.8615 - val_loss: 0.4213
Epoch 4/5
213/213 ━━━━━━━━━━━━━━━━━━━━ 22s 86ms/step - accuracy: 0.9663 - loss: 0.1264 - val_accuracy: 0.8456 - val_loss: 0.4767
Epoch 5/5
213/213 ━━━━━━━━━━━━━━━━━━━━ 19s 81ms/step - accuracy: 0.9740 - loss: 0.0944 - val_accuracy: 0.8574 - val_loss: 0.5063
67/67 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step
              precision    recall  f1-score   support

           0       0.82      0.88      0.85       672
           1       0.70      0.65      0.68       321
           2       0.93      0.91      0.92      1128

    accuracy                           0.86      2121
   macro avg       0.82      0.81      0.82      2121

Berhasil memperoleh akurasi training di 0.97 dan akurasi testing di 0.86